# Generation of Fake data 

In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import mysql.connector
fake = Faker()
Faker.seed(42)
np.random.seed(42)
random.seed(42)
print("All set")

All set


### connection 

In [14]:
conn = mysql.connector.connect(
    host="127.0.0.1",
    port=3307,
    user="sadop_user",
    password="1234",
    database="sadop_DBB"
)
cursor = conn.cursor()
print("Connected to SADOP database")

Connected to SADOP database


### Inintialization 

In [15]:
fake = Faker()

In [16]:
NUM_USERS = 20_000
NUM_ACCOUNTS = 35_000
NUM_TRANSACTIONS = 250_000
NUM_LOGS = 300_000


### 1.1 insert 10000 row fake data for users 

In [17]:
users = []

for i in range(1, NUM_USERS + 1):
    users.append({
        "user_id": i,
        "full_name": fake.name(),
        "email": fake.unique.email(),
        "country": random.choice(["DZ", "FR", "DE", "US", "UK"]),
        "signup_date": fake.date_between(start_date="-5y", end_date="today")
    })

df_users = pd.DataFrame(users)
df_users.head()


,user_id,full_name,email,country,signup_date
0,1,Charles Gomez,rebecca64@example.com,US,2024-10-17
1,2,Chelsea Lee,murphymary@example.net,UK,2022-08-20
2,3,Julie Drake,clifford77@example.org,FR,2021-09-15
3,4,Sheila Chang,becky67@example.com,FR,2025-11-24
4,5,Barry Hobbs,stevensonkaren@example.net,US,2022-12-14


### insert 20000 row of  fake data to the Accounts 

In [7]:
accounts = []

for acc_id in range(1, NUM_ACCOUNTS + 1):
    accounts.append({
        "account_id": acc_id,
        "user_id": random.randint(1, NUM_USERS),
        "account_type": random.choice(["SAVINGS", "CHECKING", "BUSINESS"]),
        "balance": round(np.random.exponential(scale=5000), 2),
        "created_at": fake.date_between(start_date="-4y", end_date="today")
    })

df_accounts = pd.DataFrame(accounts)
df_accounts.head()


,account_id,user_id,account_type,balance,created_at
0,1,15894,CHECKING,2346.34,2025-09-02
1,2,18712,CHECKING,15050.61,2024-06-15
2,3,8135,SAVINGS,6583.73,2025-02-16
3,4,5709,CHECKING,4564.71,2023-05-31
4,5,14317,SAVINGS,848.12,2024-06-26


### insert transactions

In [8]:
transactions = []

for tx_id in range(1, NUM_TRANSACTIONS + 1):
    transactions.append({
        "transaction_id": tx_id,
        "account_id": random.randint(1, NUM_ACCOUNTS),
        "amount": round(np.random.normal(100, 50), 2),
        "transaction_type": random.choice(["DEBIT", "CREDIT"]),
        "transaction_date": fake.date_between(start_date="-3y", end_date="today"),
        "status": random.choice(["SUCCESS", "FAILED"])
    })

df_transactions = pd.DataFrame(transactions)
df_transactions.head()


,transaction_id,account_id,amount,transaction_type,transaction_date,status
0,1,4627,77.47,DEBIT,2024-04-12,SUCCESS
1,2,21319,105.35,DEBIT,2025-11-19,SUCCESS
2,3,5608,61.23,CREDIT,2023-07-30,SUCCESS
3,4,7205,110.61,CREDIT,2023-02-05,SUCCESS
4,5,19080,60.02,CREDIT,2024-07-20,FAILED


### insert fake data row for the Logs 

In [9]:
logs = []

for log_id in range(1, NUM_LOGS + 1):
    logs.append({
        "log_id": log_id,
        "user_id": random.randint(1, NUM_USERS),
        "log_level": random.choice(["INFO", "WARN", "ERROR"]),
        "message": fake.text(max_nb_chars=200),
        "created_at": fake.date_time_between(start_date="-2y", end_date="now")
    })

df_logs = pd.DataFrame(logs)
df_logs.head()


,log_id,user_id,log_level,message,created_at
0,1,11014,ERROR,Event front despite brother. Meeting law also ...,2025-05-25 14:29:47
1,2,13889,INFO,No a item politics newspaper rich. Say at fill...,2024-06-28 12:24:26
2,3,42,ERROR,Condition there card leader career. Decide mod...,2025-04-20 03:02:47
3,4,13302,INFO,Among involve stuff maybe. Positive century li...,2025-07-03 03:24:40
4,5,4225,INFO,Mother edge my experience about approach visit...,2024-09-27 20:16:23


### display data and tabels 

In [11]:
print("Users:", df_users.shape)
print("Accounts:", df_accounts.shape)
print("Transactions:", df_transactions.shape)
print("Logs:", df_logs.shape)

assert df_users["user_id"].is_unique
assert df_accounts["account_id"].is_unique
assert df_transactions["transaction_id"].is_unique


Users: (20000, 5)
Accounts: (35000, 5)
Transactions: (250000, 6)
Logs: (300000, 5)


In [12]:
df_users.to_csv("../data/users.csv", index=False)
df_accounts.to_csv("../data/accounts.csv", index=False)
df_transactions.to_csv("../data/transactions.csv", index=False)
df_logs.to_csv("../data/logs.csv", index=False)

print("✅ Fake data generated and saved.")


✅ Fake data generated and saved.
